In [1]:
# !pip install altair
# !pip install altair_saver --upgrade
# !npm install -g vega-lite vega-cli canvas
# !pip install vl-convert-python --upgrade

In [2]:
# !pip install selenium --upgrade
# !apt-get install chromium-chromedriver -y

In [3]:
# !apt update
# !apt install ttf-mscorefonts-installer -y
# !apt reinstall fontconfig fontconfig-config libfontconfig1 -y

In [4]:
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Black.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Bold.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Book.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Medium.otf -P /usr/local/share/fonts

In [5]:
# !fc-cache -f

In [207]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import urllib.parse

In [208]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [230]:
SAVE = True
LOCAL = False
DARK = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [231]:
%%capture pwd
!pwd

In [232]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
uid=urllib.parse.quote(uid)
if not LOCAL:
    eco_git_home = (
        "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
    )
    vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
    colors = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
    )
    category_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
    )
    hue_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
    )
    mhue_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
    )
    div_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
    )
    config = json.loads(
        requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
    )
else:
    eco_git_home = '/'.join(pwd.stdout.split("/")[:-2])+'/'
    vega_embed = open(eco_git_home + "guidelines/html/vega-embed.html",'r').read()
    colors = json.load(
        open(eco_git_home + "guidelines/colors/eco-colors.json",'r')
    )
    category_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-category-color.json",'r')
    )
    hue_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-single-hue-color.json",'r')
    )
    mhue_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-multi-hue-color.json",'r')
    )
    div_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-diverging-color.json",'r')
    )
    config = json.load(
        open(eco_git_home + "guidelines/charts/eco-global-config.json",'r')
    )
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
mo=0.5
height = config["height"]
width = config["width"]
uid, height, width

('how-is-competition-changing-across-the-uk-economy', 300, 500)

In [233]:
def save(df, f, LOCAL):
    fc = eco_git_path + f + ".csv"
    df.to_csv("data/" + f + ".csv")
    f += local_suffix
    open("visualisation/" + f + ".html", "w").write(
        vega_embed.replace(
            "JSON_PATH", fc.replace("/data/", "/visualisation/").replace(".csv", ".json")
        )
    )
    if LOCAL:
        fc = df
    
    from IPython.display import display, HTML
    display(HTML(df.head().to_html()))
    
    readme = "## Figure " + f.replace('fig','').split('_')[0] + \
        '  \n\nData: [`csv`](data/' + f + '.csv)' +\
        '  \nGitHub: [' + f + '](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/'+uid +')'+\
        ''+\
        '  \n\n### Light theme  \n\nVersions with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local.png) [`svg`](visualisation/' + f + '_local.svg) [`json`](visualisation/' + f + '_local.json) '+ \
        '  \n (**Default**) Versions with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '.png) [`svg`](visualisation/' + f + '.svg) [`json`](visualisation/' + f + '.json)'+ \
        '  \nVersions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local_no_branding.png) [`svg`](visualisation/' + f + '_local_no_branding.svg) [`json`](visualisation/' + f + '_local_no_branding.json) '+ \
        '  \nVersions (no ECO branding) with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '_no_branding.png) [`svg`](visualisation/' + f + '_no_branding.svg) [`json`](visualisation/' + f + '_no_branding.json) '+ \
        ''+\
        '  \n\n### Dark theme  \n\nVersions with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local_dark.png) [`svg`](visualisation/' + f + '_local_dark.svg) [`json`](visualisation/' + f + '_local_dark.json) '+ \
        '  \n Versions with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '_dark.png) [`svg`](visualisation/' + f + '_dark.svg) [`json`](visualisation/' + f + '_dark.json)'+ \
        '  \nVersions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local_no_branding_dark.png) [`svg`](visualisation/' + f + '_local_no_branding_dark.svg) [`json`](visualisation/' + f + '_local_no_branding_dark.json) '+ \
        '  \nVersions (no ECO branding) with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '_no_branding_dark.png) [`svg`](visualisation/' + f + '_no_branding_dark.svg) [`json`](visualisation/' + f + '_no_branding_dark.json) '+ \
        ''+\
        '  \n\n!["' + f + '"](visualisation/' + f + '.svg "' + f + '")\n\n' +\
        '  \n\n!["' + f + '_dark"](visualisation/' + f + '_dark.svg "' + f + '")\n\n' 
    return readme, f, fc

In [234]:
def area(base,color,opacity=1):
    return base.mark_area(opacity=opacity,
    interpolate="monotone",
    line={'color':color},
    color=alt.Gradient(
        gradient='linear',
        stops=[alt.GradientStop(color='#ffffff00', offset=0.2),
               alt.GradientStop(color=color, offset=0.8)],
        x1=1, #0.8
        y1=1,
        x2=1,
        y2=0
        )
    )

In [235]:
# service_color='#d6c8da' '#e4bfe2' '#ce4b96' colors['eco-turquiose']
service_color='#b4c8d8'
def dark(f):
    configSource = "visualisation/" + f + ".json"
    config = json.loads(open(configSource, "r").read())
    config['background']=colors['eco-background']
    service_color='#b4c8d8'
    for i in config['layer']:
        if 'encoding' in i:
            for x in ['x','y']:
                if x in i['encoding']:
                    if 'axis' in i['encoding'][x]:
                        for c in ['domainColor','labelColor','tickColor','titleColor','gridColor']:
                            if c in i['encoding'][x]['axis']:
                                i['encoding'][x]['axis'][c]=service_color
        if 'mark' in i:
            if 'color' in i['mark']:
                if i['mark']['color']==colors['eco-gray']:
                    i['mark']['color']=service_color
                elif i['mark']['color']==colors['eco-blue']:
                    i['mark']['color']=colors['eco-yellow']
                elif i['mark']['color']==service_color:
                    i['mark']['color']=colors['eco-green']
                elif 'stops' in i['mark']['color']:
                    for s in i['mark']['color']['stops']:
                        if 'color' in s:
                            if s['color']==colors['eco-gray']:
                                s['color']=service_color
                            elif s['color']==colors['eco-blue']:
                                s['color']=colors['eco-yellow']
                            elif s['color']==service_color:
                                s['color']=colors['eco-green']
            if 'line' in i['mark']:
                if 'color' in i['mark']['line']:
                    if i['mark']['line']['color']==colors['eco-gray']:
                        i['mark']['line']['color']=service_color
                    elif i['mark']['line']['color']==colors['eco-blue']:
                        i['mark']['line']['color']=colors['eco-yellow']
                    elif i['mark']['line']['color']==service_color:
                        i['mark']['line']['color']=colors['eco-green']
    if 'datasets' in config:
        for i in config['datasets']:
            if 'img' in config['datasets'][i][0]:
                if 'eco-icon-dark' in config['datasets'][i][0]['img']:
                    config['datasets'][i][0]['img']=config['datasets'][i][0]['img'].replace('eco-icon-dark','eco-icon-light')
    return alt.Chart.from_dict(config) 

# Fig 1

In [236]:
df = pd.read_excel("raw/EO_figures_(1).xlsx",sheet_name='Figure 1',skiprows=4)
df

,year,cost markup index
0,1997,100.00000
1,1998,100.97960
2,1999,98.48020
3,2000,97.53168
4,2001,98.06936
5,2002,94.17562
6,2003,95.83447
7,2004,100.25640
8,2005,102.76080
9,2006,97.16475


In [237]:
readme, f, fc = save(df,"fig1_markup",LOCAL)

,year,cost markup index
0,1997,100.00000
1,1998,100.97960
2,1999,98.48020
3,2000,97.53168
4,2001,98.06936


In [238]:
xmin=1996
xmax=2022
xaxis = alt.Chart(pd.DataFrame([{'x':xmin,'y':100},{'x':xmax,'y':100}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "x:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
            format='.0f',
        ),
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="Material markup (indexed to 1997 = 100)",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            tickCount=8
        ),
        scale=alt.Scale(domain=[80, 120]),
    )
)

# Create the shaded area chart
shade = alt.Chart(pd.DataFrame([
    {'Year': 2006, 'y': 80, 'y2': 120},
    {'Year': 2008, 'y': 80, 'y2': 120}
])).mark_area(color=colors['eco-gray'],opacity=0.1).encode(x='Year:Q',y='y:Q',y2='y2:Q')
shade2 = alt.Chart(pd.DataFrame([
    {'Year': 2019, 'y': 80, 'y2': 120},
    {'Year': 2022, 'y': 80, 'y2': 120}
])).mark_area(color=colors['eco-gray'],opacity=0.1).encode(x='Year:Q',y='y:Q',y2='y2:Q')

shade_label = alt.Chart(pd.DataFrame([{
    'Year': 2007.8,  # Center of shaded area
    'y': 111.5,  # Middle of shaded area
    'text': 'Great Financial Crisis'  # Change this text
}])).mark_text(
    color=colors['eco-gray'],
    opacity=0.8,
    fontSize=12,
    align='right',
    angle=0  # Change if you want to rotate the text
).encode(
    x='Year:Q',
    y='y:Q',
    text='text:N'
)

shade2_label = alt.Chart(pd.DataFrame([{
    'Year': 2020.8,  # Center of shaded area
    'y': 116.5,  # Middle of shaded area
    'text': 'COVID-19 Pandemic'  # Change this text
}])).mark_text(
    color=colors['eco-gray'],
    opacity=0.8,
    fontSize=12,
    align='right',
    angle=0  # Change if you want to rotate the text
).encode(
    x='Year:Q',
    y='y:Q',
    text='text:N'
)

base = alt.Chart(fc).encode(x='year:Q')
line1=base.mark_line(color=colors['eco-dot']).encode(y='cost markup index:Q')

layer1 = (
    (xaxis+line1+shade+shade2+shade_label+shade2_label).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 120, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:Q',y='y:Q',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "w").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig1_markup



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 2

In [239]:
df = pd.read_excel("raw/EO_figures_(1).xlsx",sheet_name='Figure 2',skiprows=7)
df.columns=['year','entry','exit','realloc','enterprise','establish','turnover']

In [240]:
readme, f, fc = save(df,"fig2_empl",LOCAL)

,year,entry,exit,realloc,enterprise,establish,turnover
0,2004,20.27877,18.15346,40.74351,NaN,NaN,NaN
1,2005,19.26184,16.57458,36.16209,21.10340,41.40674,18.14977
2,2006,18.75529,15.63860,30.02714,20.33609,40.11448,17.32603
3,2007,20.17277,18.55052,38.55153,20.05402,38.75508,17.00511
4,2008,21.49318,20.75813,37.21509,19.03847,31.57749,14.81825


In [241]:
# Create regression data for entry rate
entry_reg = alt.Chart(df).transform_regression(
    'year', 'entry', method='linear'
).mark_line(
    strokeDash=[6, 6],
    color=colors['eco-dot'],
    opacity=0.5
).encode(x='year:Q', y='entry:Q')

# Create regression data for exit rate
exit_reg = alt.Chart(df).transform_regression(
    'year', 'exit', method='linear'
).mark_line(
    strokeDash=[6, 6],
    color=colors['eco-light-blue'],
    opacity=0.5
).encode(x='year:Q', y='exit:Q')

# Create regression data for reallocation
realloc_reg = alt.Chart(df).transform_regression(
    'year', 'realloc', method='linear'
).mark_line(
    strokeDash=[6, 6],
    color=colors['eco-turquiose'],
    opacity=0.5
).encode(x='year:Q', y='realloc:Q')


# Base chart creation
base = alt.Chart(df).properties(width=250, height=150)

# Create shaded areas for all charts
shade1 = base.mark_area(
    color=colors['eco-gray'],
    opacity=0.1
).encode(
    x='year:Q',
    y=alt.value(0),
    y2=alt.value(150)
).transform_filter(
    "datum.year >= 2006 && datum.year <= 2008"
)

shade2 = base.mark_area(
    color=colors['eco-gray'],
    opacity=0.1
).encode(
    x='year:Q',
    y=alt.value(0),
    y2=alt.value(150)
).transform_filter(
    "datum.year >= 2019 && datum.year <= 2022"
)

# Add labels for shaded areas
shade_label1 = base.mark_text(
    text='Great Financial Crisis',
    color=colors['eco-gray'],
    opacity=0.8,
    fontSize=12,
    align='right',
    angle=0
).encode(
    x=alt.value(2007.8),
    y=alt.value(df[['entry','exit']].max().max() * 0.9)
)

shade_label2 = base.mark_text(
    text='COVID-19 Pandemic',
    color=colors['eco-gray'],
    opacity=0.8,
    fontSize=12,
    align='right',
    angle=0
).encode(
    x=alt.value(2020.8),
    y=alt.value(df[['entry','exit']].max().max() * 0.95)
)

# Top left: entry and exit
chart1 = alt.layer(
    shade1,
    shade2,
    # shade_label1,
    # shade_label2,
    entry_reg,  # Add entry regression
    exit_reg,   # Add exit regression
    base.mark_line(color=colors['eco-dot']).encode(
        x=alt.X('year:Q',
            axis=alt.Axis(
                grid=False,
                titleAlign="center",
                titleAnchor="middle",
                title="",
                titleY=-15,
                titleX=207,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=10,
                domainOpacity=mo,
                tickOpacity=mo,
                labelOpacity=mo+0.2,
                titleOpacity=mo+0.3,
                titleFontSize=12,
                orient="bottom",
                labelAngle=0,
                format='.0f',
            )
        ),
        y=alt.Y('entry:Q',
            axis=alt.Axis(
                gridDash=[1,5],
                gridColor=colors["eco-gray"],
                gridOpacity=mo,
                title="Entry and exit rates (%)",
                titleX=0,
                titleY=-7,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                ticks=False,
                labelPadding=5,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                domainOpacity=mo,
                tickOpacity=mo,
                labelOpacity=mo+0.2,
                titleOpacity=mo+0.3,
                titleFontSize=12,
                format='.1f',
                tickCount=8
            )
        )
    ),
    base.mark_line(color=colors['eco-light-blue']).encode(
        x='year:Q',
        y='exit:Q'
    ),
    base.mark_text(
        text='Entry rate',
        color=colors['eco-dot'],
        align='left',
        dx=5,
        fontSize=12,dy=12
    ).transform_filter(
        "datum.year == 2021"
    ).encode(
        x='year:Q',
        y='entry:Q',
    ),
    base.mark_text(
        text='Exit rate',
        color=colors['eco-light-blue'],
        align='left',
        dx=5,
        fontSize=12
    ).transform_filter(
        "datum.year == 2021"
    ).encode(
        x='year:Q',
        y='exit:Q'
    )
)

# Top right: reallocation
chart2 = alt.layer(
    shade1,
    shade2,
    realloc_reg,  # Add reallocation regression
    # shade_label1.encode(y=alt.value(df['realloc'].max() * 0.9)),
    # shade_label2.encode(y=alt.value(df['realloc'].max() * 0.95)),
    base.mark_line(color=colors['eco-turquiose']).encode(
        x=alt.X('year:Q',
            axis=alt.Axis(
                grid=False,
                titleAlign="center",
                titleAnchor="middle",
                title="",
                titleY=-15,
                titleX=207,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=10,
                domainOpacity=mo,
                tickOpacity=mo,
                labelOpacity=mo+0.2,
                titleOpacity=mo+0.3,
                titleFontSize=12,
                orient="bottom",
                labelAngle=0,
                format='.0f',
            )
        ),
        y=alt.Y('realloc:Q',
            axis=alt.Axis(
                gridDash=[1,5],
                gridColor=colors["eco-gray"],
                gridOpacity=mo,
                title="Job reallocation rate (%)",
                titleX=0,
                titleY=-7,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                ticks=False,
                labelPadding=5,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                domainOpacity=mo,
                tickOpacity=mo,
                labelOpacity=mo+0.2,
                titleOpacity=mo+0.3,
                titleFontSize=12,
                format='.1f',
                tickCount=8
            )
        )
    ),
    # Add label for reallocation
    base.mark_text(
        text='Reallocation rate',
        color=colors['eco-turquiose'],
        align='left',
        dx=5,
        fontSize=12
    ).transform_filter(
        "datum.year == 2021"
    ).encode(
        x='year:Q',
        y='realloc:Q'
    )
)

# Bottom left: enterprise and establishment
chart3 = alt.layer(
    shade1,
    shade2,
    # shade_label1.encode(y=alt.value(df[['enterprise','establish']].max().max() * 0.9)),
    # shade_label2.encode(y=alt.value(df[['enterprise','establish']].max().max() * 0.95)),
    base.mark_line(color=colors['eco-orange']).encode(
        x=alt.X('year:Q',
            axis=alt.Axis(
                grid=False,
                titleAlign="center",
                titleAnchor="middle",
                title="",
                titleY=-15,
                titleX=207,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=10,
                domainOpacity=mo,
                tickOpacity=mo,
                labelOpacity=mo+0.2,
                titleOpacity=mo+0.3,
                titleFontSize=12,
                orient="bottom",
                labelAngle=0,
                format='.0f',
            )
        ),
        y=alt.Y('enterprise:Q',
            axis=alt.Axis(
                gridDash=[1,5],
                gridColor=colors["eco-gray"],
                gridOpacity=mo,
                title="Employment share of young firms (%)",
                titleX=0,
                titleY=-7,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                ticks=False,
                labelPadding=5,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                domainOpacity=mo,
                tickOpacity=mo,
                labelOpacity=mo+0.2,
                titleOpacity=mo+0.3,
                titleFontSize=12,
                format='.1f',
                tickCount=8
            )
        )
    ),
    base.mark_line(color=colors['eco-light-blue']).encode(
        x='year:Q',
        y='establish:Q'
    ),
    base.mark_text(
        text='Enterprise rate',
        color=colors['eco-orange'],
        align='left',
        dx=5,
        fontSize=12
    ).transform_filter(
        "datum.year == 2021"
    ).encode(
        x='year:Q',
        y='enterprise:Q'
    ),
    base.mark_text(
        text='Establishment rate',
        color=colors['eco-light-blue'],
        align='left',
        dx=5,dy=-4,
        fontSize=12
    ).transform_filter(
        "datum.year == 2021"
    ).encode(
        x='year:Q',
        y='establish:Q'
    )
)

# Bottom right: turnover
chart4 = alt.layer(
    shade1,
    shade2,
    # shade_label1.encode(y=alt.value(df['turnover'].max() * 0.9)),
    # shade_label2.encode(y=alt.value(df['turnover'].max() * 0.95)),
    base.mark_line(color=colors['eco-dot']).encode(
        x=alt.X('year:Q',
            axis=alt.Axis(
                grid=False,
                titleAlign="center",
                titleAnchor="middle",
                title="",
                titleY=-15,
                titleX=207,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=10,
                domainOpacity=mo,
                tickOpacity=mo,
                labelOpacity=mo+0.2,
                titleOpacity=mo+0.3,
                titleFontSize=12,
                orient="bottom",
                labelAngle=0,
                format='.0f',
            )
        ),
        y=alt.Y('turnover:Q',
            axis=alt.Axis(
                gridDash=[1,5],
                gridColor=colors["eco-gray"],
                gridOpacity=mo,
                title="Turnover share of young firms (%)",
                titleX=0,
                titleY=-7,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                ticks=False,
                labelPadding=5,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                domainOpacity=mo,
                tickOpacity=mo,
                labelOpacity=mo+0.2,
                titleOpacity=mo+0.3,
                titleFontSize=12,
                format='.1f',
                tickCount=8
            )
        )
    ),
    # Add label for turnover
    base.mark_text(
        text='Turnover rate',
        color=colors['eco-dot'],
        align='left',
        dx=5,
        fontSize=12
    ).transform_filter(
        "datum.year == 2021"
    ).encode(
        x='year:Q',
        y='turnover:Q'
    )
)

# Add logo
logo = alt.Chart(pd.DataFrame([{
    "x": xmax, 
    "y": 45, 
    "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"
}])).mark_image(
    width=40,
    height=40,
    align='right',
    baseline='top',
    yOffset=-33,
    opacity=mo,
    xOffset=0
).encode(x='x:Q',y='y:Q',url='img:N')

# Create the 2x2 layout
layout = alt.vconcat(
    alt.hconcat(
        chart1,
        (chart2 + logo),
        spacing=60
    ),
    alt.hconcat(
        chart3,
        chart4,
        spacing=15
    )
).configure(font='Circular Std Book').configure_view(stroke=None)

# Save and display
if DARK:
    layout.save("visualisation/" + f + "_no_branding.json")
    # layer2 = dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layout.save("visualisation/" + f + "_no_branding.json")
    layout.save("visualisation/" + f + "_no_branding.svg")
    layout.save("visualisation/" + f + "_no_branding.png")
    # if DARK:
    #     layer2.save("visualisation/" + f + "_no_branding_dark.svg")
    #     layer2.save("visualisation/" + f + "_no_branding_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layout.display()
# if DARK:
#     layer2.display()

fig2_empl



alt.VConcatChart(...)

# Fig 3

In [242]:
df = pd.read_excel("raw/EO_figures_(1).xlsx",sheet_name='Figure 3',skiprows=6)
df=df.set_index('year').stack().reset_index()
df[0]=df[0].replace('.',np.nan)
df['country']=df['level_1'].str.split('.1').str[0].str.strip()
df['kind']=df['level_1'].str.split('.1').str[1]
df=df.drop('level_1',axis=1)
df['kind']=df['kind'].fillna('creation').replace('','destruction')
df=df.rename(columns={0:'value'})
df['country']=df['country'].replace('The United Kingdom','United Kingdom')
df['country']=df['country'].replace('the Netherlands','Netherlands')

In [243]:
readme, f, fc = save(df,"fig3_job",LOCAL)

,year,value,country,kind
0,1998,NaN,Belgium,creation
1,1998,NaN,Croatia,creation
2,1998,NaN,Czech Republic,creation
3,1998,NaN,Germany,creation
4,1998,NaN,Hungary,creation


In [244]:
# Base chart creation
base = alt.Chart(df).properties(width=450, height=200)

# Create shaded areas for all charts
shade1 = base.mark_area(
    color=colors['eco-gray'],
    opacity=0.1
).encode(
    x='year:Q',
    y=alt.value(0),
    y2=alt.value(200)
).transform_filter(
    "datum.year >= 2006 && datum.year <= 2008"
)

shade2 = base.mark_area(
    color=colors['eco-gray'],
    opacity=0.1
).encode(
    x='year:Q',
    y=alt.value(0),
    y2=alt.value(200)
).transform_filter(
    "datum.year >= 2019 && datum.year <= 2022"
)

def create_country_lines(base_chart, kind_filter):
    # Other countries (thickness 1, grey)
    other_countries = base_chart.mark_line(
        strokeWidth=1,
        opacity=0.3,
        color=colors['eco-gray']
    ).transform_filter(
        f"datum.kind == '{kind_filter}' && datum.country != 'UK' && datum.country != 'Germany' && datum.country != 'Netherlands' && datum.country != 'Italy' && datum.country != 'Spain'"
    ).encode(
        x='year:Q',
        y='value:Q',
        detail='country'  # This keeps lines separate while maintaining same color
    )
    
    # Major countries (thickness 2)
    major_countries = base_chart.mark_line(
        strokeWidth=2,
        opacity=0.8
    ).transform_filter(
        f"datum.kind == '{kind_filter}' && (datum.country == 'Germany' || datum.country == 'Netherlands' || datum.country == 'Italy' || datum.country == 'Spain')"
    ).encode(
        x='year:Q',
        y='value:Q',
        color=alt.Color('country:N',
            scale=alt.Scale(
                domain=['Germany', 'Netherlands', 'Italy', 'Spain','United Kingdom','Other'],
                range=[colors['eco-dot'], colors['eco-turquiose'], colors['eco-light-blue'], colors['eco-orange'],colors['eco-purple'],colors['eco-gray']]
            ),
            legend=alt.Legend(
                title=None
            )
        )
    )
    
    # UK line (thickness 3)
    uk = base_chart.mark_line(
        strokeWidth=3,
        opacity=0.9
    ).transform_filter(
        f"datum.kind == '{kind_filter}' && datum.country == 'United Kingdom'"
    ).encode(
        x='year:Q',
        y='value:Q',
        color=alt.Color('country:N',
            scale=alt.Scale(
                domain=['United Kingdom']
            )
        )
    )
    
    # Add "Other" to legend
    other_legend = base_chart.mark_line(
        strokeWidth=1,
        opacity=0.5
    ).transform_calculate(
        country="'Other'"
    ).encode(
        x=alt.value(0),
        y=alt.value(0),
        color=alt.Color('country:N',
            scale=alt.Scale(
                domain=['Other']
            )
        )
    )
    
    return alt.layer(
        other_countries,
        major_countries,
        uk,
        other_legend
    )

# Create the charts for each kind
creation_chart = alt.layer(
    shade1,
    shade2,
    create_country_lines(base, 'creation')
).encode(
    x=alt.X('year:Q',
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
            format='.0f',
        )
    ),
    y=alt.Y('value:Q',
        axis=alt.Axis(
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="Job creation rate (indexed to 2011 = 100)",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            tickCount=8
        )
    )
)

destruction_chart = alt.layer(
    shade1,
    shade2,
    create_country_lines(base, 'destruction')
).encode(
    x=alt.X('year:Q',
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
            format='.0f',
        )
    ),
    y=alt.Y('value:Q',
        axis=alt.Axis(
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="Job destruction rate (indexed to 2011 = 100)",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            tickCount=8
        )
    )
)

# Add logo
logo = alt.Chart(pd.DataFrame([{
    "x": df['year'].max(), 
    "y": 450, 
    "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"
}])).mark_image(
    width=40,
    height=40,
    align='right',
    baseline='top',
    yOffset=-33,
    opacity=mo,
    xOffset=0
).encode(x='x:Q',y='y:Q',url='img:N')

# Create the vertical layout
layout = alt.vconcat(
    (creation_chart + logo),
    destruction_chart,
    spacing=15
).configure(font='Circular Std Book').configure_view(stroke=None)

# Save and display
if DARK:
    layout.save("visualisation/" + f + "_no_branding.json")
    # layer2 = dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layout.save("visualisation/" + f + "_no_branding.json")
    layout.save("visualisation/" + f + "_no_branding.svg")
    layout.save("visualisation/" + f + "_no_branding.png")
    # if DARK:
    #     layer2.save("visualisation/" + f + "_no_branding_dark.svg")
    #     layer2.save("visualisation/" + f + "_no_branding_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layout.display()
# if DARK:
#     layer2.display()

fig3_job



alt.VConcatChart(...)

# Post-rpocess

## Clean up `SVG`s
`base64 URI` encode images

In [245]:
from base64 import b64encode

In [246]:
from os import listdir
from os.path import isfile, join
mypath='./visualisation/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
svgs=[i for i in onlyfiles if i[-4:]=='.svg']

In [247]:
for svg in svgs:
    s=open(mypath+svg,'r').read()
    si=s.find('"image mark" xlink:href=')
    if si>-1:
        s2=s[si+25:]
        imgurl=s2[:s2.find('"')]
        if 'data:image/png;' not in imgurl:
            s1=s[:si+25]+s2.replace(imgurl,'data:image/png;base64,'+b64encode(requests.get(imgurl).content).decode('utf-8'))
            open(mypath+svg,'w').write(s1)
            print(svg)

fig1_markup.svg
fig1_markup_dark.svg
fig1_markup_local.svg
fig1_markup_local_dark.svg
fig2_empl_local_no_branding.svg
fig2_empl_no_branding.svg
fig3_job_local_no_branding.svg
fig3_job_no_branding.svg


## README

In [248]:
from IPython.display import display, Markdown

with open('README.md', 'r') as fh:
    content = fh.read()

display(Markdown(content))

## Figure 1  

Data: [`csv`](data/fig1_markup.csv)  
GitHub: [fig1_markup](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-is-competition-changing-across-the-uk-economy)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_markup_local.png) [`svg`](visualisation/fig1_markup_local.svg) [`json`](visualisation/fig1_markup_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig1_markup.png) [`svg`](visualisation/fig1_markup.svg) [`json`](visualisation/fig1_markup.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_markup_local_no_branding.png) [`svg`](visualisation/fig1_markup_local_no_branding.svg) [`json`](visualisation/fig1_markup_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig1_markup_no_branding.png) [`svg`](visualisation/fig1_markup_no_branding.svg) [`json`](visualisation/fig1_markup_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_markup_local_dark.png) [`svg`](visualisation/fig1_markup_local_dark.svg) [`json`](visualisation/fig1_markup_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig1_markup_dark.png) [`svg`](visualisation/fig1_markup_dark.svg) [`json`](visualisation/fig1_markup_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_markup_local_no_branding_dark.png) [`svg`](visualisation/fig1_markup_local_no_branding_dark.svg) [`json`](visualisation/fig1_markup_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig1_markup_no_branding_dark.png) [`svg`](visualisation/fig1_markup_no_branding_dark.svg) [`json`](visualisation/fig1_markup_no_branding_dark.json)   

!["fig1_markup"](visualisation/fig1_markup.svg "fig1_markup")

  

!["fig1_markup_dark"](visualisation/fig1_markup_dark.svg "fig1_markup")

## Figure 2  

Data: [`csv`](data/fig2_empl.csv)  
GitHub: [fig2_empl](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-is-competition-changing-across-the-uk-economy)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig2_empl_local.png) [`svg`](visualisation/fig2_empl_local.svg) [`json`](visualisation/fig2_empl_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig2_empl.png) [`svg`](visualisation/fig2_empl.svg) [`json`](visualisation/fig2_empl.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig2_empl_local_no_branding.png) [`svg`](visualisation/fig2_empl_local_no_branding.svg) [`json`](visualisation/fig2_empl_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig2_empl_no_branding.png) [`svg`](visualisation/fig2_empl_no_branding.svg) [`json`](visualisation/fig2_empl_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig2_empl_local_dark.png) [`svg`](visualisation/fig2_empl_local_dark.svg) [`json`](visualisation/fig2_empl_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig2_empl_dark.png) [`svg`](visualisation/fig2_empl_dark.svg) [`json`](visualisation/fig2_empl_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig2_empl_local_no_branding_dark.png) [`svg`](visualisation/fig2_empl_local_no_branding_dark.svg) [`json`](visualisation/fig2_empl_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig2_empl_no_branding_dark.png) [`svg`](visualisation/fig2_empl_no_branding_dark.svg) [`json`](visualisation/fig2_empl_no_branding_dark.json)   

!["fig2_empl"](visualisation/fig2_empl.svg "fig2_empl")

  

!["fig2_empl_dark"](visualisation/fig2_empl_dark.svg "fig2_empl")

## Figure 3  

Data: [`csv`](data/fig3_job.csv)  
GitHub: [fig3_job](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-is-competition-changing-across-the-uk-economy)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig3_job_local.png) [`svg`](visualisation/fig3_job_local.svg) [`json`](visualisation/fig3_job_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig3_job.png) [`svg`](visualisation/fig3_job.svg) [`json`](visualisation/fig3_job.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig3_job_local_no_branding.png) [`svg`](visualisation/fig3_job_local_no_branding.svg) [`json`](visualisation/fig3_job_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig3_job_no_branding.png) [`svg`](visualisation/fig3_job_no_branding.svg) [`json`](visualisation/fig3_job_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig3_job_local_dark.png) [`svg`](visualisation/fig3_job_local_dark.svg) [`json`](visualisation/fig3_job_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig3_job_dark.png) [`svg`](visualisation/fig3_job_dark.svg) [`json`](visualisation/fig3_job_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig3_job_local_no_branding_dark.png) [`svg`](visualisation/fig3_job_local_no_branding_dark.svg) [`json`](visualisation/fig3_job_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig3_job_no_branding_dark.png) [`svg`](visualisation/fig3_job_no_branding_dark.svg) [`json`](visualisation/fig3_job_no_branding_dark.json)   

!["fig3_job"](visualisation/fig3_job.svg "fig3_job")

  

!["fig3_job_dark"](visualisation/fig3_job_dark.svg "fig3_job")

